In [156]:
# Check the versions of libraries
 
# Python version
import sys
print('Python: {}'.format(sys.version))
# scipy
import scipy
print('scipy: {}'.format(scipy.__version__))
# numpy
import numpy
print('numpy: {}'.format(numpy.__version__))
# matplotlib
import matplotlib
print('matplotlib: {}'.format(matplotlib.__version__))
# pandas
# scikit-learn
import pandas as pd
import sklearn
print('sklearn: {}'.format(sklearn.__version__))
import seaborn as sns
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn import metrics
#!pip install sklearn.cross_validation
#from sklearn.cross_validation import train_test_split
import numpy as np

# allow plots to appear directly in the notebook
%matplotlib inline

Python: 3.6.10 |Anaconda, Inc.| (default, May  7 2020, 19:46:08) [MSC v.1916 64 bit (AMD64)]
scipy: 1.5.0
numpy: 1.19.5
matplotlib: 3.3.2
sklearn: 0.24.1


In [157]:
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import seaborn as sns
...

Ellipsis

In [160]:
# Load dataset
url = "Resources/train_loandata.csv"
#names =['FL_DATE','OP_CARRIER','ORIGIN','DEST','DEP_TIME','DEP_DELAY', 'ARR_TIME','ARR_DELAY',' AIR_TIME', 'DISTANCE', 'class']

dataset = read_csv(url)
#names=names)

In [164]:
dataset.head(2)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N


In [165]:
newdata=dataset
#['Loan_ID','Gender','Married','Dependents','Education','Self_Employed','ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term','Credit_History','Property_Area']
newdata.dropna(inplace=True)
#newdata.head()
#print(newdata)

In [166]:
newdata.dtypes

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [167]:
# shape
print(newdata.shape)

(480, 13)


In [168]:
newdata.to_csv("Resources/newdata.csv")

In [7]:
#Statistical summary

In [169]:
# descriptions
print(newdata.describe())

       ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
count       480.000000         480.000000  480.000000        480.000000   
mean       5364.231250        1581.093583  144.735417        342.050000   
std        5668.251251        2617.692267   80.508164         65.212401   
min         150.000000           0.000000    9.000000         36.000000   
25%        2898.750000           0.000000  100.000000        360.000000   
50%        3859.000000        1084.500000  128.000000        360.000000   
75%        5852.500000        2253.250000  170.000000        360.000000   
max       81000.000000       33837.000000  600.000000        480.000000   

       Credit_History  
count      480.000000  
mean         0.854167  
std          0.353307  
min          0.000000  
25%          1.000000  
50%          1.000000  
75%          1.000000  
max          1.000000  


In [32]:
#Group by Airline
# class distribution
#print(newdata.groupby('OP_CARRIER').size())

In [11]:
#delay_ds = ori_gp['DEP_DELAY'].mean()

In [ ]:
#newdata.OP_CARRIER.value_counts()

In [170]:
categorical_cols = ['Gender','Married','Dependents','Education','Self_Employed','Property_Area','Loan_Status']
from sklearn.preprocessing import LabelEncoder
# instantiate labelencoder object
le = LabelEncoder()

# apply le on categorical feature columns
newdata[categorical_cols] = newdata[categorical_cols].apply(lambda col: le.fit_transform(col))    
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()

#One-hot-encode the categorical columns.
#Unfortunately outputs an array instead of dataframe.
array_hot_encoded = ohe.fit_transform(newdata[categorical_cols])

#Convert it to df
data_hot_encoded = pd.DataFrame(array_hot_encoded, index=newdata.index)

#Extract only the columns that didnt need to be encoded
data_other_cols = newdata.drop(columns=categorical_cols)

#Concatenate the two dataframes : 
data_out = pd.concat([data_hot_encoded, data_other_cols], axis=1)

In [171]:
#data_out.head()

In [173]:
# One hot encoding - to convert categorical data to continuous
cat_vars = ['Gender','Married','Dependents','Education','Self_Employed','Property_Area','Loan_Status']
df2 = newdata[cat_vars].astype(str)
df3 = pd.get_dummies(df2)
df3.head(5)

,Gender_0,Gender_1,Married_0,Married_1,Dependents_0,Dependents_1,Dependents_2,Dependents_3,Education_0,Education_1,Self_Employed_0,Self_Employed_1,Property_Area_0,Property_Area_1,Property_Area_2,Loan_Status_0,Loan_Status_1
1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,1,0
2,0,1,0,1,1,0,0,0,1,0,0,1,0,0,1,0,1
3,0,1,0,1,1,0,0,0,0,1,1,0,0,0,1,0,1
4,0,1,1,0,1,0,0,0,1,0,1,0,0,0,1,0,1
5,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1


In [174]:
#Extract only the columns that didnt need to be encoded
data_other_cols = newdata.drop(columns=categorical_cols)
data_num_df=data_other_cols.to_csv("Resources/data_num.csv")

In [43]:
#Concatenate the two dataframes : 
#data_out = pd.concat([data_hot_encoded, data_other_cols], axis=1)

In [52]:
#data_out.head()

In [177]:
df3.to_csv("Resources/catdat.csv")

In [176]:
import pandas as pd
catdat_df = pd.read_csv("Resources/catdat.csv",
                         keep_default_na=False, na_values=[""])
catdat_df

,Unnamed: 0,Gender_0,Gender_1,Married_0,Married_1,Dependents_0,Dependents_1,Dependents_2,Dependents_3,Education_0,Education_1,Self_Employed_0,Self_Employed_1,Property_Area_0,Property_Area_1,Property_Area_2,Loan_Status_0,Loan_Status_1
0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,1,0
1,2,0,1,0,1,1,0,0,0,1,0,0,1,0,0,1,0,1
2,3,0,1,0,1,1,0,0,0,0,1,1,0,0,0,1,0,1
3,4,0,1,1,0,1,0,0,0,1,0,1,0,0,0,1,0,1
4,5,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,609,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0,0,1
476,610,0,1,0,1,0,0,0,1,1,0,1,0,1,0,0,0,1
477,611,0,1,0,1,0,1,0,0,1,0,1,0,0,0,1,0,1
478,612,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,1


In [78]:
newdata_df= pd.read_csv("Resources/newdata.csv",
                         keep_default_na=False, na_values=[""])
newdata_df.head(2)

,Unnamed: 0,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban


In [178]:
full_set_df=pd.merge(newdata_df, catdat_df, left_index=True, right_index=True)
full_set_df.to_csv("Resources/full_set.csv")
full_set_df

,Unnamed: 0_x,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,...,Dependents_3,Education_0,Education_1,Self_Employed_0,Self_Employed_1,Property_Area_0,Property_Area_1,Property_Area_2,Loan_Status_0,Loan_Status_1
0,0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,...,0,1,0,1,0,1,0,0,1,0
1,1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,...,0,1,0,0,1,0,0,1,0,1
2,2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,...,0,0,1,1,0,0,0,1,0,1
3,4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,...,0,1,0,1,0,0,0,1,0,1
4,5,LP001054,Male,Yes,0,Not Graduate,Yes,2165,3422,152.0,...,0,1,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,361,LP002969,Male,Yes,1,Graduate,No,2269,2167,99.0,...,0,1,0,1,0,0,1,0,1,0
285,362,LP002971,Male,Yes,3+,Not Graduate,Yes,4009,1777,113.0,...,0,1,0,1,0,0,1,0,0,1
286,363,LP002975,Male,Yes,0,Graduate,No,4158,709,115.0,...,0,1,0,1,0,1,0,0,1,0
287,365,LP002986,Male,Yes,0,Graduate,No,5000,2393,158.0,...,0,1,0,0,1,0,1,0,0,1


In [194]:
Full_set3_df=pd.read_csv("Resources/full_set3.csv")
Full_set3_df.head(1)

,Loan-yes,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,...,1Dependents,2Dependents,3Dependents,Graduate,non_grad,Not self_empl,Self_Employed_1,Rural,Semiurban,Urban
0,1,Male,Yes,0,Graduate,No,5720,0,110,360,...,0,0,0,1,0,1,0,0,0,1


In [196]:
Full_set2_df=Full_set3_df.drop(['Gender','Married','Dependents','Education','Self_Employed','Property_Area'],axis=1)
Full_set2_df.head()
Full_set2_df.to_csv("Resources/Full_set2.csv")

Full_set2_df.head()
                                     

,Loan-yes,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Unnamed: 0_y,Female,Male,Unmarried,...,1Dependents,2Dependents,3Dependents,Graduate,non_grad,Not self_empl,Self_Employed_1,Rural,Semiurban,Urban
0,1,5720,0,110,360,1,0,0,1,0,...,0,0,0,1,0,1,0,0,0,1
1,0,3076,1500,126,360,1,1,0,1,0,...,1,0,0,1,0,1,0,0,0,1
2,0,5000,1800,208,360,1,2,0,1,0,...,0,1,0,1,0,1,0,0,0,1
3,0,3276,0,78,360,1,4,0,1,1,...,0,0,0,0,1,1,0,0,0,1
4,0,2165,3422,152,360,1,5,0,1,0,...,0,0,0,0,1,0,1,0,0,1


In [185]:
Full_set2_df.columns

Index(['Loan-yes', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Unnamed: 0_y', 'Female', 'Male',
       'Unmarried', 'Married.1', '0Dependents', '1Dependents', '2Dependents',
       '3Dependents', 'Graduate', 'non_grad', 'Not self_empl',
       'Self_Employed_1', 'Rural', 'Semiurban', 'Urban'],
      dtype='object')

In [186]:
X = Full_set2_df[['ApplicantIncome', 'Credit_History','Female','Male','Unmarried','Married.1','1Dependents','2Dependents','3Dependents','Graduate','non_grad','Self_Employed_1','Rural','Semiurban','Urban']]
y = Full_set2_df['LoanAmount']

In [187]:
from sklearn import linear_model

In [188]:
regr = linear_model.LinearRegression()
regr.fit(X, y)

LinearRegression()

In [189]:
print(regr.coef_)

[ 5.32259811e-03 -1.26553398e+01 -2.30982533e-01  2.30982533e-01
 -8.22334695e+00  8.22334695e+00  7.15554941e-01  1.48976164e+01
  1.92229935e-01  1.00745252e+01 -1.00745252e+01  9.18165331e-01
  3.81758996e+00 -4.90050948e-01 -3.32753901e+00]


In [190]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
#from sklearn.linear_model import pvalues
# create X and y
feature_cols = ['ApplicantIncome', 'Credit_History','Female','Male','Unmarried','Married.1','1Dependents','2Dependents','3Dependents','Graduate','non_grad','Self_Employed_1','Rural','Semiurban','Urban']
X = Full_set2_df[feature_cols]
y = Full_set2_df.LoanAmount

# instantiate and fit
lm2 = LinearRegression()
lm2.fit(X, y)

# print the coefficients
list(zip(feature_cols, lm2.coef_))

[('ApplicantIncome', 0.005322598113966024),
 ('Credit_History', -12.655339801896373),
 ('Female', -0.23098253255807272),
 ('Male', 0.230982532558067),
 ('Unmarried', -8.22334694962331),
 ('Married.1', 8.223346949623311),
 ('1Dependents', 0.7155549407359599),
 ('2Dependents', 14.89761638428901),
 ('3Dependents', 0.19222993481347567),
 ('Graduate', 10.074525188027605),
 ('non_grad', -10.074525188027604),
 ('Self_Employed_1', 0.9181653314251923),
 ('Rural', 3.8175899568485465),
 ('Semiurban', -0.4900509480620882),
 ('Urban', -3.3275390087864563)]

In [193]:
lm2.score(x,Y)

NameError: name 'x' is not defined